In [1]:
import asyncio

loop = asyncio.get_event_loop()

In [2]:
import unittest
from unittest import IsolatedAsyncioTestCase
from sdjwt.sdjwt import (
    generate_did_key_from_seed,
    create_w3c_vc_sd_jwt_for_data_attributes,
)
from sdjwt.didkey import DIDKey
from sdjwt.adapter import DataAttribute, DataAttributesAdapter





def create_w3c_vc_jwt_for_passport(didkey: DIDKey):
    credential_id = "urn:did:abc"
    credential_type = ["Passport"]
    credential_context = ["https://www.w3.org/2018/credentials/v1"]
    credential_schema = [
        {
            "id": "https://api-conformance.ebsi.eu/trusted-schemas-registry/v2/schemas/z3MgUFUkb722uq4x3dv5yAJmnNmzDFeK5UC8x83QoeLJM",
            "type": "FullJsonSchemaValidator2021",
        }
    ]
    data_attributes = [
        DataAttribute(name="section1.name", value="John Doe"),
        DataAttribute(name="section1.age", value=24),
        DataAttribute(name="section2.address.state", value="KL"),
        DataAttribute(name="section2.address.city", value="Kochi"),
    ]

    kid = "did:key:issuer_did#issuer_did"
    jti = credential_id
    iss = "did:key:issuer_did"
    sub = "did:key:datawallet_did"
    to_be_issued_credential = create_w3c_vc_sd_jwt_for_data_attributes(
        credential_id=credential_id,
        credential_type=credential_type,
        credential_context=credential_context,
        data_attributes=data_attributes,
        credential_status=None,
        terms_of_use=None,
        credential_schema=credential_schema,
        kid=kid,
        jti=jti,
        iss=iss,
        sub=sub,
        key=didkey.private_key,
        credential_issuer=didkey.generate()[0],
        limited_disclosure=True
    )

    return to_be_issued_credential

async def main():
    crypto_seed = "helloworld"
    key_did = await generate_did_key_from_seed(crypto_seed)
    key_did.generate()
    cred = create_w3c_vc_jwt_for_passport(didkey=key_did)
    print(cred)


await main()


eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6aXNzdWVyX2RpZCNpc3N1ZXJfZGlkIiwidHlwIjoiSldUIn0.eyJleHAiOjE3MTQ0Nzg0MjIsImlhdCI6MTcxNDQ3NDgyMiwiaXNzIjoiZGlkOmtleTppc3N1ZXJfZGlkIiwianRpIjoidXJuOmRpZDphYmMiLCJuYmYiOjE3MTQ0NzQ4MjIsInN1YiI6ImRpZDprZXk6ZGF0YXdhbGxldF9kaWQiLCJ2YyI6eyJAY29udGV4dCI6WyJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy92MSJdLCJjcmVkZW50aWFsU2NoZW1hIjpbeyJpZCI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvdHJ1c3RlZC1zY2hlbWFzLXJlZ2lzdHJ5L3YyL3NjaGVtYXMvejNNZ1VGVWtiNzIydXE0eDNkdjV5QUptbk5tekRGZUs1VUM4eDgzUW9lTEpNIiwidHlwZSI6IkZ1bGxKc29uU2NoZW1hVmFsaWRhdG9yMjAyMSJ9XSwiY3JlZGVudGlhbFN1YmplY3QiOnsiX3NkIjpbInJ5NWZTckVHaGlKaTRSZkEzdGQ5WmZXUE1mendoUGNfNVd4Z2xqTlZ4MjAiLCI1aTJZZ0dDSU5EckE2TVJqRVhxODFYSEJoZ2kxRXgwbnVBcFJBSEZCQXJBIl19LCJleHBpcmF0aW9uRGF0ZSI6IjIwMjQtMDQtMzBUMTI6MDA6MjJaIiwiaWQiOiJ1cm46ZGlkOmFiYyIsImlzc3VhbmNlRGF0ZSI6IjIwMjQtMDQtMzBUMTE6MDA6MjJaIiwiaXNzdWVkIjoiMjAyNC0wNC0zMFQxMTowMDoyMloiLCJpc3N1ZXIiOiJkaWQ6a2V5OnoyalF2SHVzVnZETHA4TGI1V1FuVUVva3R4a0VIZm9RMjkxRVc2WlBrSldH

In [3]:
def update_credential(credential, disclosure_mapping):
    def update_value(obj, path):
        for key in path[:-1]:
            obj = obj.setdefault(key, {})
        obj[path[-1]] = {"_sd": []}

    def iterate_mapping(obj, path):
        for key, value in obj.items():
            if isinstance(value, dict):
                new_path = path + [key]
                if "limitedDisclosure" in value and value["limitedDisclosure"]:
                    update_value(credential, new_path)
                iterate_mapping(value, new_path)

    iterate_mapping(disclosure_mapping, [])


credential = {
    "credentialSubject": {
        "Vehicle identifier": "1111",
        "section1": {"address": {"city": ""}, "occupation": ""},
        "section2": {
            "nestedfield": [
                {"value": "True"},
                {"value": "True"},
            ]
        },
    },
    "type": ["VerifiableCredential", "ParkingTicket"],
}

disclosure_mapping = {
    "credentialSubject": {
        "section1": {
            "address": {"limitedDisclosure": True},
            "occupation": {"limitedDisclosure": True},
        },
    }
}

update_credential(credential, disclosure_mapping)
print(credential)

{'credentialSubject': {'Vehicle identifier': '1111', 'section1': {'address': {'_sd': []}, 'occupation': {'_sd': []}}, 'section2': {'nestedfield': [{'value': 'True'}, {'value': 'True'}]}}, 'type': ['VerifiableCredential', 'ParkingTicket']}


In [1]:
import unittest
from unittest import IsolatedAsyncioTestCase
from sdjwt.sdjwt import (
    generate_did_key_from_seed,
    create_w3c_vc_jwt_with_disclosure_mapping,
)
from sdjwt.didkey import DIDKey
from sdjwt.adapter import DataAttribute, DataAttributesAdapter


def create_w3c_vc_jwt_for_passport(didkey: DIDKey):
    credential_id = "urn:did:abc"
    credential_type = ["Passport"]
    credential_context = ["https://www.w3.org/2018/credentials/v1"]
    credential_schema = [
        {
            "id": "https://api-conformance.ebsi.eu/trusted-schemas-registry/v2/schemas/z3MgUFUkb722uq4x3dv5yAJmnNmzDFeK5UC8x83QoeLJM",
            "type": "FullJsonSchemaValidator2021",
        }
    ]
    credential_subject = {
        "credentialSubject": {
            "id": None,
            "section1": {
                "dateBirth": "1968-12-29",
                "forenames": "Charlotte",
                "nationalities": ["SE"],
                "personalIdentificationNumber": "19681229-1412",
                "placeBirth": {
                    "countryCode": "SE",
                    "region": "Stockholm",
                    "town": "Stockholm",
                },
                "sex": "Female",
                "stateOfResidenceAddress": {
                    "countryCode": "SE",
                    "postCode": "418 78",
                    "streetNo": "Gunnar Engellaus vag 8, 91 1B",
                    "town": "Stockholm",
                },
                "stateOfStayAddress": {
                    "countryCode": "SE",
                    "postCode": "418 78",
                    "streetNo": "Gunnar Engellaus vag 8, 91 1B",
                    "town": "Stockholm",
                },
                "surname": "Anderson",
                "surnameAtBirth": "Anderson",
            },
            "section2": {
                "certificateForDurationActivity": True,
                "determinationProvisional": False,
                "endingDate": "2024-07-03",
                "memberStateWhichLegislationApplies": "IT",
                "startingDate": "2023-09-21",
                "transitionRulesApplyAsEC8832004": False,
            },
            "section3": {
                "civilAndEmployedSelfEmployed": False,
                "civilServant": False,
                "contractStaff": False,
                "employedAndSelfEmployed": False,
                "employedTwoOrMoreStates": False,
                "exception": False,
                "exceptionDescription": "",
                "flightCrewMember": False,
                "mariner": False,
                "postedEmployedPerson": False,
                "postedSelfEmployedPerson": True,
                "selfEmployedTwoOrMoreStates": False,
                "workingInStateUnder21": False,
            },
            "section4": {
                "employee": False,
                "employerSelfEmployedActivityCodes": ["1889113244"],
                "nameBusinessName": "Volvo",
                "registeredAddress": {
                    "countryCode": "SE",
                    "postCode": "418 78",
                    "streetNo": "Gunnar Engellaus vu00e4g 8, 164 A",
                    "town": "Goteborg",
                },
                "selfEmployedActivity": True,
            },
            "section5": {
                "noFixedAddress": False,
                "workPlaceAddresses": [
                    {
                        "address": {
                            "countryCode": "IT",
                            "postCode": "34132",
                            "streetNo": "Piazza Duca degli Abruzzi 2, 440",
                            "town": "Trieste",
                        },
                        "seqno": 1,
                    }
                ],
                "workPlaceNames": [
                    {
                        "companyNameVesselName": "Assicurazioni Generali S.p.A",
                        "seqno": 1,
                    }
                ],
            },
            "section6": {
                "address": {
                    "countryCode": "BE",
                    "postCode": "1000",
                    "streetNo": "Main Street 1",
                    "town": "Brussels",
                },
                "date": "2023-09-07",
                "email": "info@nssi-be.eu",
                "institutionID": "NSSI-BE-01",
                "name": "National Social Security Office",
                "officeFaxNo": "0800 98765",
                "officePhoneNo": "0800 12345",
                "signature": "Official signature",
            },
        },
    }

    disclosure_mapping = {
        "credentialSubject": {
            "section1": {"nationalities": {"limitedDisclosure": True}},
            "section4": {
                "employerSelfEmployedActivityCodes": {"limitedDisclosure": True},
                "registeredAddress": {
                    "postCode": {"limitedDisclosure": True},
                    "streetNo": {"limitedDisclosure": True},
                },
            },
            "section3": {"limitedDisclosure": True},
            "section2": {"limitedDisclosure": True},
            "section5": {"limitedDisclosure": True},
            "section6": {"limitedDisclosure": True},
        }
    }
    kid = "did:key:issuer_did#issuer_did"
    jti = credential_id
    iss = "did:key:issuer_did"
    sub = "did:key:datawallet_did"
    to_be_issued_credential = create_w3c_vc_jwt_with_disclosure_mapping(
        credential_id=credential_id,
        credential_type=credential_type,
        credential_context=credential_context,
        credential_subject=credential_subject,
        credential_status=None,
        terms_of_use=None,
        credential_schema=credential_schema,
        kid=kid,
        jti=jti,
        iss=iss,
        sub=sub,
        key=didkey.private_key,
        credential_issuer=didkey.generate()[0],
        disclosure_mapping=None,
    )

    return to_be_issued_credential


async def main():
    crypto_seed = "helloworld"
    key_did = await generate_did_key_from_seed(crypto_seed)
    key_did.generate()
    cred = create_w3c_vc_jwt_for_passport(didkey=key_did)
    print(cred)
    # for disclosure in cred.split("~")[1:]:
    #     print(disclosure)


await main()

eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6aXNzdWVyX2RpZCNpc3N1ZXJfZGlkIiwidHlwIjoiSldUIn0.eyJleHAiOjE3MTQ0ODU3NDgsImlhdCI6MTcxNDQ4MjE0OCwiaXNzIjoiZGlkOmtleTppc3N1ZXJfZGlkIiwianRpIjoidXJuOmRpZDphYmMiLCJuYmYiOjE3MTQ0ODIxNDgsInN1YiI6ImRpZDprZXk6ZGF0YXdhbGxldF9kaWQiLCJ2YyI6eyJAY29udGV4dCI6WyJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy92MSJdLCJjcmVkZW50aWFsU2NoZW1hIjpbeyJpZCI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvdHJ1c3RlZC1zY2hlbWFzLXJlZ2lzdHJ5L3YyL3NjaGVtYXMvejNNZ1VGVWtiNzIydXE0eDNkdjV5QUptbk5tekRGZUs1VUM4eDgzUW9lTEpNIiwidHlwZSI6IkZ1bGxKc29uU2NoZW1hVmFsaWRhdG9yMjAyMSJ9XSwiY3JlZGVudGlhbFN1YmplY3QiOnsiaWQiOm51bGwsInNlY3Rpb24xIjp7ImRhdGVCaXJ0aCI6IjE5NjgtMTItMjkiLCJmb3JlbmFtZXMiOiJDaGFybG90dGUiLCJuYXRpb25hbGl0aWVzIjpbIlNFIl0sInBlcnNvbmFsSWRlbnRpZmljYXRpb25OdW1iZXIiOiIxOTY4MTIyOS0xNDEyIiwicGxhY2VCaXJ0aCI6eyJjb3VudHJ5Q29kZSI6IlNFIiwicmVnaW9uIjoiU3RvY2tob2xtIiwidG93biI6IlN0b2NraG9sbSJ9LCJzZXgiOiJGZW1hbGUiLCJzdGF0ZU9mUmVzaWRlbmNlQWRkcmVzcyI6eyJjb3VudHJ5Q29kZSI6IlNFIiwicG9zdENvZGUiOiI0MTgg